# Assignment 2

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
from patsy import dmatrices
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [9]:
# DATA IMPORT - FROM GITHUB
data_raw = pd.read_csv('https://raw.githubusercontent.com/thimaipham/2.CEU_DA3_Assignment2/main/listings.csv')

In [8]:
# Removing unnecessary columns to streamline the dataset for this assignment
# List of columns to be dropped based on the description and analysis
columns_to_drop = [
    'listing_url', 'scrape_id', 'picture_url', 'host_url', 'host_thumbnail_url',
    'host_picture_url', 'host_name', 'host_since', 'host_about', 'name',
    'description', 'neighborhood_overview', 'host_listings_count', 'license'
]

# Dropping the columns from the DataFrame
data_cleaned = data_raw.drop(columns=columns_to_drop, errors='ignore')

# Display the shape of the new DataFrame to confirm the drop
data_cleaned.info


NameError: name 'data_raw' is not defined

Do dữ liệu khá nhiều cột mà theo tôi không cần thiết sử dụng trong assignment này vì vậy tôi sẽ loại bỏ 1 số cột để làm cho file dữ liệu trở nên gọn gàng hơn